# OOpenCV、Dlib、MediaPipeを用いた画像認識 ｰ深層学習ｰ

ここでは、カメラから取得した映像を用いて画像認識を行い、
必要な情報を取得するための流れを学ぶことで、
画像認識をビジネス現場で応用するイメージをつかみます。

### MediaPipeを用いた骨格検出

このPythonスクリプトは、OpenCVとOpenPoseを使用して、リアルタイムの人体ポーズ推定を行います。主な機能は以下の通りです：

1. ウェブカメラからのビデオ入力を取得します。
2. MediaPipeのPoseソリューションを使用して、人体の33個の主要な関節点（ランドマーク）を検出します。
3. 検出された骨格を可視化し、関節間の接続線を描画します。
4. 肘と膝の関節角度を計算して表示します。

In [6]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipeのポーズ検出モジュールを初期化
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# ポーズ検出器を設定（静的画像モードはFalse、最大1人検出）
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

def calculate_angle(a, b, c):
    """
    3つの点から角度を計算する関数
    a: 始点の座標 [x, y]
    b: 中心点の座標 [x, y]（角度を測る点）
    c: 終点の座標 [x, y]
    戻り値: 角度（度数法）
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    # ベクトルBA、BCを計算
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # 180度を超える場合は、小さい方の角度を採用
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

# Webカメラを起動（0は既定のカメラ）
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("カメラから映像を取得できません")
        break
    
    # BGRからRGBに変換（MediaPipeはRGB形式を使用）
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False
    
    # ポーズ検出を実行
    results = pose.process(image_rgb)
    
    # 画像を書き込み可能に戻し、RGBからBGRに変換
    image_rgb.flags.writeable = True
    image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    
    # ランドマークが検出された場合
    if results.pose_landmarks:
        # 骨格を描画
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
        )
        
        # 画像のサイズを取得
        height, width, _ = image.shape
        
        # ランドマークを取得
        landmarks = results.pose_landmarks.landmark
        
        # 左肘の角度を計算（肩-肘-手首）
        try:
            # 左肩、左肘、左手首の座標を取得
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * width,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * height]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x * width,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y * height]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x * width,
                         landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y * height]
            
            # 角度を計算
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            
            # 角度を画面に表示
            cv2.putText(image, f"Left Elbow: {int(left_elbow_angle)} deg",
                       (int(left_elbow[0]), int(left_elbow[1])),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
        
        # 右肘の角度を計算（肩-肘-手首）
        try:
            # 右肩、右肘、右手首の座標を取得
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * width,
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * height]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x * width,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y * height]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x * width,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y * height]
            
            # 角度を計算
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            # 角度を画面に表示
            cv2.putText(image, f"Right Elbow: {int(right_elbow_angle)} deg",
                       (int(right_elbow[0]), int(right_elbow[1])),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
        
        # 左膝の角度を計算（腰-膝-足首）
        try:
            # 左腰、左膝、左足首の座標を取得
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * width,
                       landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * height]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * width,
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * height]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x * width,
                         landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y * height]
            
            # 角度を計算
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            
            # 角度を画面に表示
            cv2.putText(image, f"Left Knee: {int(left_knee_angle)} deg",
                       (int(left_knee[0]), int(left_knee[1])),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2, cv2.LINE_AA)
        except:
            pass
        
        # 右膝の角度を計算（腰-膝-足首）
        try:
            # 右腰、右膝、右足首の座標を取得
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x * width,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y * height]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x * width,
                         landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y * height]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].x * width,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y * height]
            
            # 角度を計算
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            
            # 角度を画面に表示
            cv2.putText(image, f"Right Knee: {int(right_knee_angle)} deg",
                       (int(right_knee[0]), int(right_knee[1])),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2, cv2.LINE_AA)
        except:
            pass
    
    # 操作説明を表示
    cv2.putText(image, "Press 'q' to quit", (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # 結果を表示
    cv2.imshow('MediaPipe Pose Detection', image)
    
    # 'q'キーで終了
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# リソースを解放
pose.close()
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

### Darknet (Yolov5)

YOLOv5は、Ultralytics社によって開発された、YOLOv3を改良したリアルタイム物体検出アルゴリズムです。高速かつ高精度な検出が可能です。軽量なモデルから高精度なモデルまで、サイズの異なる複数のモデルが提供されており、簡単にカスタマイズや学習ができるため、様々な用途に適しています。YOLOv5に関する情報は豊富で、自作の物体検出器を構築する際には、v5の利用をおすすめします。

**本プログラムを実行する際には、OpenCVのバージョンアップデートが必要なため、GitHub上の講義資料に従って専用の環境構築を行ってください。**

このプログラムはOpenCVの[参考資料](https://learnopencv.com/object-detection-using-yolov5-and-opencv-dnn-in-c-and-python/)を元に作成されています．

このプログラムを実行するためには、学習済みのDNNモデルが必要です。Google Colab上で以下のコードを順に実行して、`yolov5s.onnx`を作成してダウンロードしてください。または、[ここから](https://kutc-my.sharepoint.com/:u:/g/personal/t160024_kansai-u_ac_jp/ETb0zCwQVPlPo5-dPP1HitIB1erpb53UGZzaZkHQgyQuhA?e=CAGSPG)ダウンロードしてください。保存先は、フォルダ`det/`です。

```
!git clone https://github.com/ultralytics/yolov5
%cd /content/yolov5/
!pip install -r requirements.txt
!wget https://github.com/ultralytics/YOLOv5/releases/download/v6.1/YOLOv5s.pt
!python export.py --weights YOLOv5s.pt --opset 12 --include onnx
```

その他の参考資料

[https://github.com/ultralytics/yolov5?tab=readme-ov-file](https://github.com/ultralytics/yolov5?tab=readme-ov-file)

[https://docs.ultralytics.com/ja](https://docs.ultralytics.com/ja)

[https://docs.ultralytics.com/ja/yolov5/tutorials/model_export/](https://docs.ultralytics.com/ja/yolov5/tutorials/model_export/)

[https://qiita.com/02130515/items/cfbb9dcd8291418476ab](https://qiita.com/02130515/items/cfbb9dcd8291418476ab)

In [ ]:
import cv2
import numpy as np

# 定数設定
INPUT_WIDTH = 640  # 入力画像の幅
INPUT_HEIGHT = 640  # 入力画像の高さ
SCORE_THRESHOLD = 0.5  # スコアのしきい値
NMS_THRESHOLD = 0.45  # 非最大抑制のしきい値
CONFIDENCE_THRESHOLD = 0.45  # 信頼度のしきい値

# テキスト表示に関する設定
FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.7  # フォントのサイズ
THICKNESS = 1  # テキストの太さ

# 色の設定
BLACK  = (0, 0, 0)
BLUE   = (255, 178, 50)
YELLOW = (0, 255, 255)
RED = (0, 0, 255)

# ラベルを描画する関数
def draw_label(input_image, label, left, top):
    """ラベル（テキスト）を指定の位置に描画する"""
    
    # テキストのサイズを取得
    text_size = cv2.getTextSize(label, FONT_FACE, FONT_SCALE, THICKNESS)
    dim, baseline = text_size[0], text_size[1]
    # テキストの背景用の黒い矩形を描画
    cv2.rectangle(input_image, (left, top), (left + dim[0], top + dim[1] + baseline), BLACK, cv2.FILLED)
    # 矩形の中にテキストを描画
    cv2.putText(input_image, label, (left, top + dim[1]), FONT_FACE, FONT_SCALE, YELLOW, THICKNESS, cv2.LINE_AA)

# 前処理を行う関数
def pre_process(input_image, net):
    """画像をネットワークに入力するための前処理"""
    
    # 画像をBLOB形式に変換
    blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WIDTH, INPUT_HEIGHT), [0,0,0], 1, crop=False)

    # ネットワークに入力をセット
    net.setInput(blob)

    # 出力レイヤーの名前を取得し、フォワードパスで結果を取得
    output_layers = net.getUnconnectedOutLayersNames()
    outputs = net.forward(output_layers)

    return outputs

# 後処理を行う関数
def post_process(input_image, outputs, classes):
    """ネットワークの出力結果に基づいて後処理を行い、物体を検出して描画する"""
    
    # クラスID、信頼度、バウンディングボックスを保存するリスト
    class_ids = []
    confidences = []
    boxes = []

    # 出力結果の行数を取得
    rows = outputs[0].shape[1]

    # 入力画像のサイズを取得
    image_height, image_width = input_image.shape[:2]

    # リサイズ係数の計算
    x_factor = image_width / INPUT_WIDTH
    y_factor = image_height / INPUT_HEIGHT

    # 各検出結果を処理
    for r in range(rows):
        row = outputs[0][0][r]
        confidence = row[4]  # 信頼度を取得

        # 信頼度が閾値を超えた場合のみ処理
        if confidence >= CONFIDENCE_THRESHOLD:
            classes_scores = row[5:]  # 各クラスのスコアを取得

            # スコアが最も高いクラスIDを取得
            class_id = np.argmax(classes_scores)

            # スコアがしきい値を超える場合のみ処理
            if (classes_scores[class_id] > SCORE_THRESHOLD):
                confidences.append(confidence)
                class_ids.append(class_id)

                # バウンディングボックスの中心座標、幅、高さを取得
                cx, cy, w, h = row[0], row[1], row[2], row[3]

                # 左上の座標を計算
                left = int((cx - w/2) * x_factor)
                top = int((cy - h/2) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)

                # バウンディングボックスを保存
                box = np.array([left, top, width, height])
                boxes.append(box)

    # 非最大抑制を使用して重複するボックスを除去
    indices = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    for i in indices:
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        # バウンディングボックスを描画
        cv2.rectangle(input_image, (left, top), (left + width, top + height), BLUE, 3*THICKNESS)
        # ラベルを描画
        label = "{}:{:.2f}".format(classes[class_ids[i]], confidences[i])
        draw_label(input_image, label, left, top)

    return input_image

# カメラ映像を使ったリアルタイム物体検出
def run_real_time_detection():
    """カメラを使用してリアルタイムで物体検出を行う"""

    # クラス名を読み込み
    classesFile = "det/coco.names"
    classes = None
    with open(classesFile, 'rt') as f:
        classes = f.read().rstrip('\n').split('\n')

    # YOLOv5モデルを読み込む
    modelWeights = "det/yolov5s.onnx"
    net = cv2.dnn.readNet(modelWeights)

    # デフォルトカメラ（0）を使用して映像を取得
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

    while True:
        # カメラからフレームをキャプチャ
        ret, frame = cap.read()
        if not ret:
            print("カメラからフレームを取得できませんでした。")
            break

        # 前処理を行い、検出を実行
        detections = pre_process(frame, net)

        # 検出結果に基づいて後処理を行い、フレームに描画
        processed_frame = post_process(frame.copy(), detections, classes)

        # 検出結果を別ウィンドウで表示
        cv2.imshow('YOLOv5 Object Detection', processed_frame)

        # 'q'キーを押すと終了
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 終了時にカメラを解放し、ウィンドウを閉じる
    cap.release()
    cv2.destroyAllWindows()

# リアルタイム物体検出を実行
run_real_time_detection()